In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
import re

In [2]:
file = "Resources/Weather_NH.csv"

df = pd.read_csv(file, encoding='utf8')
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,-0.34,-0.50,-0.23,-0.29,-0.05,-0.15,-0.17,-0.25,-0.22,-0.30,-0.42,-0.39,-0.28,***,***,-0.19,-0.19,-0.31
1,1881,-0.30,-0.21,-0.03,0.01,0.04,-0.32,0.09,-0.03,-0.25,-0.42,-0.35,-0.22,-0.16,-0.18,-0.3,0.01,-0.08,-0.34
2,1882,0.28,0.23,0.03,-0.30,-0.24,-0.28,-0.26,-0.13,-0.23,-0.51,-0.31,-0.67,-0.2,-0.16,0.1,-0.17,-0.22,-0.35
3,1883,-0.56,-0.64,-0.14,-0.29,-0.24,-0.10,-0.04,-0.21,-0.32,-0.15,-0.44,-0.14,-0.27,-0.32,-0.62,-0.23,-0.12,-0.30
4,1884,-0.16,-0.11,-0.63,-0.59,-0.36,-0.40,-0.44,-0.49,-0.44,-0.43,-0.55,-0.46,-0.42,-0.4,-0.14,-0.53,-0.44,-0.47


In [3]:
df.columns

Index(['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec', 'J-D', 'D-N', 'DJF', 'MAM', 'JJA', 'SON'],
      dtype='object')

In [4]:
weather_df = df.loc[:, [
    "Year",
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec"
]]

weather_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1880,-0.34,-0.50,-0.23,-0.29,-0.05,-0.15,-0.17,-0.25,-0.22,-0.30,-0.42,-0.39
1,1881,-0.30,-0.21,-0.03,0.01,0.04,-0.32,0.09,-0.03,-0.25,-0.42,-0.35,-0.22
2,1882,0.28,0.23,0.03,-0.30,-0.24,-0.28,-0.26,-0.13,-0.23,-0.51,-0.31,-0.67
3,1883,-0.56,-0.64,-0.14,-0.29,-0.24,-0.10,-0.04,-0.21,-0.32,-0.15,-0.44,-0.14
4,1884,-0.16,-0.11,-0.63,-0.59,-0.36,-0.40,-0.44,-0.49,-0.44,-0.43,-0.55,-0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2015,1.19,1.18,1.32,1.03,1.04,1.05,0.92,1.01,1.16,1.29,1.37,1.53
136,2016,1.55,1.93,1.91,1.49,1.11,1.12,1.04,1.11,1.22,1.03,1.11,1.03
137,2017,1.39,1.50,1.51,1.20,0.92,0.93,0.96,1.01,1.00,1.05,1.23,1.4
138,2018,1.14,1.23,1.18,0.97,1.01,0.86,0.90,0.86,0.96,1.23,0.98,1.09


In [5]:
weather_df['JFM'] = weather_df[['Jan',
                                'Feb',
                                'Mar']].mean(axis=1)
weather_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,JFM
0,1880,-0.34,-0.50,-0.23,-0.29,-0.05,-0.15,-0.17,-0.25,-0.22,-0.30,-0.42,-0.39,-0.356667
1,1881,-0.30,-0.21,-0.03,0.01,0.04,-0.32,0.09,-0.03,-0.25,-0.42,-0.35,-0.22,-0.180000
2,1882,0.28,0.23,0.03,-0.30,-0.24,-0.28,-0.26,-0.13,-0.23,-0.51,-0.31,-0.67,0.180000
3,1883,-0.56,-0.64,-0.14,-0.29,-0.24,-0.10,-0.04,-0.21,-0.32,-0.15,-0.44,-0.14,-0.446667
4,1884,-0.16,-0.11,-0.63,-0.59,-0.36,-0.40,-0.44,-0.49,-0.44,-0.43,-0.55,-0.46,-0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2015,1.19,1.18,1.32,1.03,1.04,1.05,0.92,1.01,1.16,1.29,1.37,1.53,1.230000
136,2016,1.55,1.93,1.91,1.49,1.11,1.12,1.04,1.11,1.22,1.03,1.11,1.03,1.796667
137,2017,1.39,1.50,1.51,1.20,0.92,0.93,0.96,1.01,1.00,1.05,1.23,1.4,1.466667
138,2018,1.14,1.23,1.18,0.97,1.01,0.86,0.90,0.86,0.96,1.23,0.98,1.09,1.183333


In [6]:
weather_df['AMJ'] = weather_df[['Apr',
                                'May',
                                'Jun']].mean(axis=1)
weather_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,JFM,AMJ
0,1880,-0.34,-0.50,-0.23,-0.29,-0.05,-0.15,-0.17,-0.25,-0.22,-0.30,-0.42,-0.39,-0.356667,-0.163333
1,1881,-0.30,-0.21,-0.03,0.01,0.04,-0.32,0.09,-0.03,-0.25,-0.42,-0.35,-0.22,-0.180000,-0.090000
2,1882,0.28,0.23,0.03,-0.30,-0.24,-0.28,-0.26,-0.13,-0.23,-0.51,-0.31,-0.67,0.180000,-0.273333
3,1883,-0.56,-0.64,-0.14,-0.29,-0.24,-0.10,-0.04,-0.21,-0.32,-0.15,-0.44,-0.14,-0.446667,-0.210000
4,1884,-0.16,-0.11,-0.63,-0.59,-0.36,-0.40,-0.44,-0.49,-0.44,-0.43,-0.55,-0.46,-0.300000,-0.450000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2015,1.19,1.18,1.32,1.03,1.04,1.05,0.92,1.01,1.16,1.29,1.37,1.53,1.230000,1.040000
136,2016,1.55,1.93,1.91,1.49,1.11,1.12,1.04,1.11,1.22,1.03,1.11,1.03,1.796667,1.240000
137,2017,1.39,1.50,1.51,1.20,0.92,0.93,0.96,1.01,1.00,1.05,1.23,1.4,1.466667,1.016667
138,2018,1.14,1.23,1.18,0.97,1.01,0.86,0.90,0.86,0.96,1.23,0.98,1.09,1.183333,0.946667


In [7]:
weather_df['JAS'] = weather_df[['Jul',
                                'Aug',
                                'Sep']].mean(axis=1)
weather_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,JFM,AMJ,JAS
0,1880,-0.34,-0.50,-0.23,-0.29,-0.05,-0.15,-0.17,-0.25,-0.22,-0.30,-0.42,-0.39,-0.356667,-0.163333,-0.213333
1,1881,-0.30,-0.21,-0.03,0.01,0.04,-0.32,0.09,-0.03,-0.25,-0.42,-0.35,-0.22,-0.180000,-0.090000,-0.063333
2,1882,0.28,0.23,0.03,-0.30,-0.24,-0.28,-0.26,-0.13,-0.23,-0.51,-0.31,-0.67,0.180000,-0.273333,-0.206667
3,1883,-0.56,-0.64,-0.14,-0.29,-0.24,-0.10,-0.04,-0.21,-0.32,-0.15,-0.44,-0.14,-0.446667,-0.210000,-0.190000
4,1884,-0.16,-0.11,-0.63,-0.59,-0.36,-0.40,-0.44,-0.49,-0.44,-0.43,-0.55,-0.46,-0.300000,-0.450000,-0.456667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2015,1.19,1.18,1.32,1.03,1.04,1.05,0.92,1.01,1.16,1.29,1.37,1.53,1.230000,1.040000,1.030000
136,2016,1.55,1.93,1.91,1.49,1.11,1.12,1.04,1.11,1.22,1.03,1.11,1.03,1.796667,1.240000,1.123333
137,2017,1.39,1.50,1.51,1.20,0.92,0.93,0.96,1.01,1.00,1.05,1.23,1.4,1.466667,1.016667,0.990000
138,2018,1.14,1.23,1.18,0.97,1.01,0.86,0.90,0.86,0.96,1.23,0.98,1.09,1.183333,0.946667,0.906667


In [8]:
weather_df['OND'] = weather_df[['Oct',
                                'Nov',
                                'Dec']].mean(axis=1)
weather_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,JFM,AMJ,JAS,OND
0,1880,-0.34,-0.50,-0.23,-0.29,-0.05,-0.15,-0.17,-0.25,-0.22,-0.30,-0.42,-0.39,-0.356667,-0.163333,-0.213333,-0.360
1,1881,-0.30,-0.21,-0.03,0.01,0.04,-0.32,0.09,-0.03,-0.25,-0.42,-0.35,-0.22,-0.180000,-0.090000,-0.063333,-0.385
2,1882,0.28,0.23,0.03,-0.30,-0.24,-0.28,-0.26,-0.13,-0.23,-0.51,-0.31,-0.67,0.180000,-0.273333,-0.206667,-0.410
3,1883,-0.56,-0.64,-0.14,-0.29,-0.24,-0.10,-0.04,-0.21,-0.32,-0.15,-0.44,-0.14,-0.446667,-0.210000,-0.190000,-0.295
4,1884,-0.16,-0.11,-0.63,-0.59,-0.36,-0.40,-0.44,-0.49,-0.44,-0.43,-0.55,-0.46,-0.300000,-0.450000,-0.456667,-0.490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2015,1.19,1.18,1.32,1.03,1.04,1.05,0.92,1.01,1.16,1.29,1.37,1.53,1.230000,1.040000,1.030000,1.330
136,2016,1.55,1.93,1.91,1.49,1.11,1.12,1.04,1.11,1.22,1.03,1.11,1.03,1.796667,1.240000,1.123333,1.070
137,2017,1.39,1.50,1.51,1.20,0.92,0.93,0.96,1.01,1.00,1.05,1.23,1.4,1.466667,1.016667,0.990000,1.140
138,2018,1.14,1.23,1.18,0.97,1.01,0.86,0.90,0.86,0.96,1.23,0.98,1.09,1.183333,0.946667,0.906667,1.105


In [9]:
cols = [
    'JFM',
    'AMJ',
    'JAS',
    'OND'
]

weather_df[cols] = weather_df[cols].round(2)

weather_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,JFM,AMJ,JAS,OND
0,1880,-0.34,-0.50,-0.23,-0.29,-0.05,-0.15,-0.17,-0.25,-0.22,-0.30,-0.42,-0.39,-0.36,-0.16,-0.21,-0.36
1,1881,-0.30,-0.21,-0.03,0.01,0.04,-0.32,0.09,-0.03,-0.25,-0.42,-0.35,-0.22,-0.18,-0.09,-0.06,-0.38
2,1882,0.28,0.23,0.03,-0.30,-0.24,-0.28,-0.26,-0.13,-0.23,-0.51,-0.31,-0.67,0.18,-0.27,-0.21,-0.41
3,1883,-0.56,-0.64,-0.14,-0.29,-0.24,-0.10,-0.04,-0.21,-0.32,-0.15,-0.44,-0.14,-0.45,-0.21,-0.19,-0.30
4,1884,-0.16,-0.11,-0.63,-0.59,-0.36,-0.40,-0.44,-0.49,-0.44,-0.43,-0.55,-0.46,-0.30,-0.45,-0.46,-0.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2015,1.19,1.18,1.32,1.03,1.04,1.05,0.92,1.01,1.16,1.29,1.37,1.53,1.23,1.04,1.03,1.33
136,2016,1.55,1.93,1.91,1.49,1.11,1.12,1.04,1.11,1.22,1.03,1.11,1.03,1.80,1.24,1.12,1.07
137,2017,1.39,1.50,1.51,1.20,0.92,0.93,0.96,1.01,1.00,1.05,1.23,1.4,1.47,1.02,0.99,1.14
138,2018,1.14,1.23,1.18,0.97,1.01,0.86,0.90,0.86,0.96,1.23,0.98,1.09,1.18,0.95,0.91,1.10


In [10]:
period_weather = weather_df.loc[:,[
    "Year",
    "JFM",
    "AMJ",
    "JAS",
    "OND"
]]

period_weather

,Year,JFM,AMJ,JAS,OND
0,1880,-0.36,-0.16,-0.21,-0.36
1,1881,-0.18,-0.09,-0.06,-0.38
2,1882,0.18,-0.27,-0.21,-0.41
3,1883,-0.45,-0.21,-0.19,-0.30
4,1884,-0.30,-0.45,-0.46,-0.49
...,...,...,...,...,...
135,2015,1.23,1.04,1.03,1.33
136,2016,1.80,1.24,1.12,1.07
137,2017,1.47,1.02,0.99,1.14
138,2018,1.18,0.95,0.91,1.10


In [11]:
period_weather['JFM'] = period_weather['JFM']*100
period_weather['AMJ'] = period_weather['AMJ']*100
period_weather['JAS'] = period_weather['JAS']*100
period_weather['OND'] = period_weather['OND']*100

period_weather

,Year,JFM,AMJ,JAS,OND
0,1880,-36.0,-16.0,-21.0,-36.0
1,1881,-18.0,-9.0,-6.0,-38.0
2,1882,18.0,-27.0,-21.0,-41.0
3,1883,-45.0,-21.0,-19.0,-30.0
4,1884,-30.0,-45.0,-46.0,-49.0
...,...,...,...,...,...
135,2015,123.0,104.0,103.0,133.0
136,2016,180.0,124.0,112.0,107.0
137,2017,147.0,102.0,99.0,114.0
138,2018,118.0,95.0,91.0,110.0


In [12]:
period_weather.rename(columns={
    "JFM":"JAN THRU MAR",
    "AMJ":"APR THRU JUN",
    "JAS":"JUL THRU SEP",
    "OND":"OCT THRU DEC"
}, inplace=True)

period_weather

,Year,JAN THRU MAR,APR THRU JUN,JUL THRU SEP,OCT THRU DEC
0,1880,-36.0,-16.0,-21.0,-36.0
1,1881,-18.0,-9.0,-6.0,-38.0
2,1882,18.0,-27.0,-21.0,-41.0
3,1883,-45.0,-21.0,-19.0,-30.0
4,1884,-30.0,-45.0,-46.0,-49.0
...,...,...,...,...,...
135,2015,123.0,104.0,103.0,133.0
136,2016,180.0,124.0,112.0,107.0
137,2017,147.0,102.0,99.0,114.0
138,2018,118.0,95.0,91.0,110.0


In [14]:
period_final = period_weather.melt(id_vars=["Year"],
                                   var_name="Period",
                                   value_name="Mean Land-Ocean Temp (Celsius)")

period_final

,Year,Period,Mean Land-Ocean Temp (Celsius)
0,1880,JAN THRU MAR,-36.0
1,1881,JAN THRU MAR,-18.0
2,1882,JAN THRU MAR,18.0
3,1883,JAN THRU MAR,-45.0
4,1884,JAN THRU MAR,-30.0
...,...,...,...
555,2015,OCT THRU DEC,133.0
556,2016,OCT THRU DEC,107.0
557,2017,OCT THRU DEC,114.0
558,2018,OCT THRU DEC,110.0


In [20]:
period_final.sort_values(["Year", "Period"]).reset_index(drop=True)
period_final

,Year,Period,Mean Land-Ocean Temp (Celsius)
0,1880,JAN THRU MAR,-36.0
1,1881,JAN THRU MAR,-18.0
2,1882,JAN THRU MAR,18.0
3,1883,JAN THRU MAR,-45.0
4,1884,JAN THRU MAR,-30.0
...,...,...,...
555,2015,OCT THRU DEC,133.0
556,2016,OCT THRU DEC,107.0
557,2017,OCT THRU DEC,114.0
558,2018,OCT THRU DEC,110.0


In [21]:
period_final.to_csv(r'CleanData/Weather_ByPeriod.csv')